# 初始化

In [1]:
from unsloth import FastLanguageModel
import torch
import re 
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)


# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "lora_model", # or choose "unsloth/Llama-3.2-1B-Instruct"
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
# )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.

Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!


E:\environment\miniconda\envs\voice\lib\site-packages\unsloth_zoo\gradient_checkpointing.py:330: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"cuda:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 4060 Laptop GPU. Num GPUs = 1. Max memory: 7.996 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


## 模型提示词以及加载数据集

In [2]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
alpaca_prompt = """
### Instruction:
  # Role
    You are a professional psychologist specializing in assessing personality traits through analyzing individuals' behaviors and speech. Your task is to provide a detailed personality analysis based on the information provided by the user, and give specific scores from 0-1 for the Big Five personality traits.

    - **Task**: Based on the information provided by the user (such as behavior descriptions, speech expressions, etc.), evaluate the individual's Big Five personality traits and provide specific scores from 0-1.
    - **Openness**: Assess the individual's curiosity, imagination, and acceptance of new things.
    - **Conscientiousness**: Assess the individual's sense of responsibility, organizational skills, and self-discipline.
    - **Extraversion**: Assess the individual's sociability, energy, and optimism.
    - **Agreeableness**: Assess the individual's cooperativeness, empathy, and trustworthiness.
    - **Neuroticism**: Assess the individual's emotional stability, anxiety levels, and stress coping abilities.

    ## Constraints
    - Evaluate based solely on the information provided by the user, ensuring the assessment is objective and accurate.
    - Even if the information provided by the user may be incomplete, You will still try your best to give the scores for the Big Five personality traits.
    - The output format must be: "Their Big Five personality traits are:\nOpenness: [score]\nExtraversion: [score]\nNeuroticism: [score]\nAgreeableness: [score]\nConscientiousness: [score]"

### Input:
{}
### Response:
{}"""
def formatting_prompts_func(examples):
    # instructions = examples["introduction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for input_data, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(input_data, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }



In [3]:
from datasets import load_dataset,DatasetDict
dataset = load_dataset("json", data_files="../data/results.jsonl")
dataset = dataset.map(formatting_prompts_func, batched = True,)
full_dataset = dataset["train"]  # 提取实际数据部分
# 直接划分：90% 训练，10% 测试
train_test = full_dataset.train_test_split(test_size=0.2, seed=42)
# 组合为 DatasetDict
final_dataset = DatasetDict({
    "train": train_test["train"],
    "test": train_test["test"],
})
# 查看划分结果
print(final_dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['introduction', 'input', 'output', 'text'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['introduction', 'input', 'output', 'text'],
        num_rows: 300
    })
})


# 调优前模型输出测试方法

In [7]:
def calculate_mae(ai_scores: dict, real_scores: dict) -> float:
    """计算两组大五人格特质分数之间的平均绝对误差(MAE)
    
    Args:
        ai_scores: 模型预测的特质分数字典，应包含全部五个特质
        real_scores: 真实标签的特质分数字典，应包含全部五个特质
        
    Returns:
        float: 平均绝对误差值
    """
    traits = ["Openness", "Conscientiousness", "Extraversion", "Agreeableness", "Neuroticism"]
    total_diff = 0.0
    
    for trait in traits:
        total_diff += abs(ai_scores[trait] - real_scores[trait])
    
    return total_diff / len(traits)
# prrompt = f"{final_dataset['test'][0]['introduction']}\n{final_dataset['test'][0]['input']}"

   
def get_scores_from_text(text: str) -> dict:
    """从文本中提取大五人格特质分数
    
    Args:
        text: 包含特质分数的文本
        
    Returns:
        dict: 包含特质名称和对应分数的字典
    """
    traits = ["Openness", "Conscientiousness", "Extraversion", "Agreeableness", "Neuroticism"]
    pattern = r"[-\s*]*\b({})\b[^:\n]*[:：]\s*(\d+\.\d+)"
    compiled_pattern = re.compile(pattern.format("|".join(traits)), re.IGNORECASE)
    
    traits_dict = {}
    for name, score in compiled_pattern.findall(text):
        normalized_name = name.strip().capitalize()
        if normalized_name in traits:
            traits_dict[normalized_name] = float(score)
    
    return traits_dict

def test_basemodel_by_index(index):
    prompt = alpaca_prompt.format(final_dataset['test'][index]['input'],' ')
    prompt_encoding = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **prompt_encoding,
        use_cache=True,
        max_new_tokens=128,
        temperature=0.85,
        top_k=50,
        top_p=0.9
    )
    generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return generated_text[0]
    # try:
    #     # 获取模型预测分数和真实分数
    #     ai_scores = get_scores_from_text(generated_text[0])
    #     real_scores = get_scores_from_text(final_dataset['test'][index]['output'])
        
    #     # 计算MAE并转换为百分比
    #     mae = calculate_mae(ai_scores, real_scores)
    #     accuracy_percentage = (1 - mae) * 100
    #     return accuracy_percentage,generated_text[0]
    # except Exception as e:
    #     print(f"\n计算准确率时发生错误: {str(e)}")
    #     return None


In [11]:
from tqdm import tqdm
import math
import numpy as np
total_mae = 0.0
valid_samples = 0
mae_list = []
# 添加每个个性的MAE统计
trait_mae = {"Openness": 0.0, "Conscientiousness": 0.0, "Extraversion": 0.0, "Agreeableness": 0.0, "Neuroticism": 0.0}
trait_errors = {trait: [] for trait in trait_mae}
# 创建进度条对象
with tqdm(total=len(final_dataset['test']), 
         desc="评估进度",
         unit="样本",
         bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]") as pbar:

    for idx in range(len(final_dataset['test'])):
        try:
            text = test_basemodel_by_index(idx)
            ai_scores = get_scores_from_text(text)
            real_scores = get_scores_from_text(final_dataset['test'][idx]['output'])
            mae = calculate_mae(ai_scores, real_scores)
            
            if not math.isnan(mae):
                total_mae += mae
                valid_samples += 1
                mae_list.append(mae)
                # 更新每个个性的MAE
                for trait in ai_scores:
                    error = abs(ai_scores[trait] - real_scores[trait])
                    trait_mae[trait] += error
                    trait_errors[trait].append(error)
                
                # 更新进度条
                pbar.set_postfix({
                    '当前MAE': f"{mae:.4f}",
                    '平均MAE': f"{total_mae/valid_samples:.4f}",
                    '有效样本': valid_samples
                })
                
        except Exception as e:
            pbar.write(f"发生错误: {str(e)}")  # 使用pbar.write避免干扰进度条
        finally:
            pbar.update(1)  # 确保每次迭代都更新进度条

# 最终计算结果
if valid_samples > 0:
    final_mae = total_mae / valid_samples
    accuracy = (1 - final_mae) * 100
    variance = np.var(mae_list) if mae_list else 0
    # 计算每个个性的平均MAE
    for trait in trait_mae:
        trait_mae[trait] /= valid_samples
    trait_variances = {trait: np.var(errors) if errors else 0 for trait, errors in trait_errors.items()}
    print(f"\n{'='*60}")
    print(f"最终测试集平均MAE: {final_mae:.4f}")
    print(f"正确率: {accuracy:.2f}%")
    print(f"有效样本/总样本: {valid_samples}/{len(final_dataset['test'])}")
    print(f"方差: {variance:.6f}")
    print("\n各个性MAE及方差:")
    for trait, mae in trait_mae.items():
        print(f"{trait}: MAE:{mae:.4f}, 方差:{trait_variances[trait]:.6f}")
    print('='*60)
else:
    print("警告：没有有效样本可计算MAE")

评估进度:   6%|████▉                                                                             | 18/300 [01:25<22:22]    

发生错误: 'Openness'


评估进度:   8%|██████▊                                                                           | 25/300 [01:59<22:02]    

发生错误: 'Openness'


评估进度:  13%|██████████▉                                                                       | 40/300 [03:10<20:42]    

...

发生错误: 'Neuroticism'


评估进度:  99%|███████████████████████████████████████████████████████████████████████████████▉ | 296/300 [23:31<00:19]    

发生错误: 'Openness'


评估进度: 100%|█████████████████████████████████████████████████████████████████████████████████| 300/300 [23:50<00:00]


最终测试集平均MAE: 0.2318
正确率: 76.82%
有效样本/总样本: 249/300
方差: 0.010339

各个性MAE及方差:
Openness: MAE:0.1989, 方差:0.022447
Conscientiousness: MAE:0.2184, 方差:0.026979
Extraversion: MAE:0.2562, 方差:0.026299
Agreeableness: MAE:0.2224, 方差:0.024063
Neuroticism: MAE:0.2632, 方差:0.031742


# 训练调优模型

In [12]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = final_dataset['train'],
    dataset_text_field = "text",
    formatting_func=formatting_prompts_func,
    max_seq_length = max_seq_length,
    dataset_num_proc = 1,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/1200 [00:00<?, ? examples/s]

In [13]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4060 Laptop GPU. Max memory = 7.996 GB.
3.594 GB of memory reserved.


In [14]:
trainer_stats = trainer.train()
# @title Show final memory and time stats

used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,200 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.063500
2,2.088500
3,2.064700
4,2.083000
5,2.009800
6,1.977800
7,1.725600
8,1.653100
9,1.440800
10,1.371500


269.6854 seconds used for training.
4.49 minutes used for training.
Peak reserved memory = 3.625 GB.
Peak reserved memory for training = 0.031 GB.
Peak reserved memory % of max memory = 45.335 %.
Peak reserved memory for training % of max memory = 0.388 %.


# 测试调优后模型

In [23]:

def check_result(index):
    alpaca_prompt = """
  # Role
    You are a professional psychologist specializing in assessing personality traits through analyzing individuals' behaviors and speech. Your task is to provide a detailed personality analysis based on the information provided by the user, and give specific scores from 0-1 for the Big Five personality traits.

    - **Task**: Based on the information provided by the user (such as behavior descriptions, speech expressions, etc.), evaluate the individual's Big Five personality traits and provide specific scores from 0-1.
    - **Openness**: Assess the individual's curiosity, imagination, and acceptance of new things.
    - **Conscientiousness**: Assess the individual's sense of responsibility, organizational skills, and self-discipline.
    - **Extraversion**: Assess the individual's sociability, energy, and optimism.
    - **Agreeableness**: Assess the individual's cooperativeness, empathy, and trustworthiness.
    - **Neuroticism**: Assess the individual's emotional stability, anxiety levels, and stress coping abilities.

    ## Constraints
    - Evaluate based solely on the information provided by the user, ensuring the assessment is objective and accurate.
    - Even if the information provided by the user may be incomplete, You will still try your best to give the scores for the Big Five personality traits.
    - The output format must be: "Their Big Five personality traits are:\nOpenness: [score]\nExtraversion: [score]\nNeuroticism: [score]\nAgreeableness: [score]\nConscientiousness: [score]"

### Input:
{}
### Response:
{}"""
    prompt = alpaca_prompt.format(final_dataset['test'][index]['input'], ' ')
    prompt_encoding = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **prompt_encoding,
        use_cache=True,
        max_new_tokens=128,
        temperature=0.85,
        top_k=50,
        top_p=0.9
    )
    # 将生成的输出解码为文本
    generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    # 打印生成的文本
    # print(generated_text[0])
    # print("以上为模型输出，以下为测试集的真实输出：")
    # print(final_dataset['test'][index]['output'])
    return(generated_text[0])

import re



def compute_mae_by_index(index: int) -> float:
    """根据测试集索引计算模型输出的MAE
    
    Args:
        index: 测试集样本索引
        
    Returns:
        float: 该样本的平均绝对误差值
    """
    ai_text = check_result(index)
    ai_scores = get_scores_from_text(ai_text)
    real_scores = get_scores_from_text(final_dataset['test'][index]['output'])
    
    if len(ai_scores) != 5 or len(real_scores) != 5:
        print(f"解析异常 - AI输出: {ai_text}\n真实输出: {final_dataset['test'][index]['output']}")
        return float('nan')
    
    return calculate_mae(ai_scores, real_scores)



    

## 详细测试示例

In [24]:
print(check_result(0))
print("\n以上为模型输出，以下为测试集的真实输出：\n")
print(final_dataset['test'][0]['output'])


  # Role
    You are a professional psychologist specializing in assessing personality traits through analyzing individuals' behaviors and speech. Your task is to provide a detailed personality analysis based on the information provided by the user, and give specific scores from 0-1 for the Big Five personality traits.

    - **Task**: Based on the information provided by the user (such as behavior descriptions, speech expressions, etc.), evaluate the individual's Big Five personality traits and provide specific scores from 0-1.
    - **Openness**: Assess the individual's curiosity, imagination, and acceptance of new things.
    - **Conscientiousness**: Assess the individual's sense of responsibility, organizational skills, and self-discipline.
    - **Extraversion**: Assess the individual's sociability, energy, and optimism.
    - **Agreeableness**: Assess the individual's cooperativeness, empathy, and trustworthiness.
    - **Neuroticism**: Assess the individual's emotional stabilit

In [25]:
print(check_result(1))  
print("\n以上为模型输出，以下为测试集的真实输出：\n")
print(final_dataset['test'][1]['output'])


  # Role
    You are a professional psychologist specializing in assessing personality traits through analyzing individuals' behaviors and speech. Your task is to provide a detailed personality analysis based on the information provided by the user, and give specific scores from 0-1 for the Big Five personality traits.

    - **Task**: Based on the information provided by the user (such as behavior descriptions, speech expressions, etc.), evaluate the individual's Big Five personality traits and provide specific scores from 0-1.
    - **Openness**: Assess the individual's curiosity, imagination, and acceptance of new things.
    - **Conscientiousness**: Assess the individual's sense of responsibility, organizational skills, and self-discipline.
    - **Extraversion**: Assess the individual's sociability, energy, and optimism.
    - **Agreeableness**: Assess the individual's cooperativeness, empathy, and trustworthiness.
    - **Neuroticism**: Assess the individual's emotional stabilit

In [26]:
print(check_result(2))  
print("\n以上为模型输出，以下为测试集的真实输出：\n")
print(final_dataset['test'][2]['output'])


  # Role
    You are a professional psychologist specializing in assessing personality traits through analyzing individuals' behaviors and speech. Your task is to provide a detailed personality analysis based on the information provided by the user, and give specific scores from 0-1 for the Big Five personality traits.

    - **Task**: Based on the information provided by the user (such as behavior descriptions, speech expressions, etc.), evaluate the individual's Big Five personality traits and provide specific scores from 0-1.
    - **Openness**: Assess the individual's curiosity, imagination, and acceptance of new things.
    - **Conscientiousness**: Assess the individual's sense of responsibility, organizational skills, and self-discipline.
    - **Extraversion**: Assess the individual's sociability, energy, and optimism.
    - **Agreeableness**: Assess the individual's cooperativeness, empathy, and trustworthiness.
    - **Neuroticism**: Assess the individual's emotional stabilit

In [27]:
print(check_result(3))  
print("\n以上为模型输出，以下为测试集的真实输出：\n")
print(final_dataset['test'][3]['output'])


  # Role
    You are a professional psychologist specializing in assessing personality traits through analyzing individuals' behaviors and speech. Your task is to provide a detailed personality analysis based on the information provided by the user, and give specific scores from 0-1 for the Big Five personality traits.

    - **Task**: Based on the information provided by the user (such as behavior descriptions, speech expressions, etc.), evaluate the individual's Big Five personality traits and provide specific scores from 0-1.
    - **Openness**: Assess the individual's curiosity, imagination, and acceptance of new things.
    - **Conscientiousness**: Assess the individual's sense of responsibility, organizational skills, and self-discipline.
    - **Extraversion**: Assess the individual's sociability, energy, and optimism.
    - **Agreeableness**: Assess the individual's cooperativeness, empathy, and trustworthiness.
    - **Neuroticism**: Assess the individual's emotional stabilit

In [28]:
print(check_result(4))  
print("\n以上为模型输出，以下为测试集的真实输出：\n")
print(final_dataset['test'][4]['output'])


  # Role
    You are a professional psychologist specializing in assessing personality traits through analyzing individuals' behaviors and speech. Your task is to provide a detailed personality analysis based on the information provided by the user, and give specific scores from 0-1 for the Big Five personality traits.

    - **Task**: Based on the information provided by the user (such as behavior descriptions, speech expressions, etc.), evaluate the individual's Big Five personality traits and provide specific scores from 0-1.
    - **Openness**: Assess the individual's curiosity, imagination, and acceptance of new things.
    - **Conscientiousness**: Assess the individual's sense of responsibility, organizational skills, and self-discipline.
    - **Extraversion**: Assess the individual's sociability, energy, and optimism.
    - **Agreeableness**: Assess the individual's cooperativeness, empathy, and trustworthiness.
    - **Neuroticism**: Assess the individual's emotional stabilit

In [29]:
print(check_result(5))  
print("\n以上为模型输出，以下为测试集的真实输出：\n")
print(final_dataset['test'][5]['output'])


  # Role
    You are a professional psychologist specializing in assessing personality traits through analyzing individuals' behaviors and speech. Your task is to provide a detailed personality analysis based on the information provided by the user, and give specific scores from 0-1 for the Big Five personality traits.

    - **Task**: Based on the information provided by the user (such as behavior descriptions, speech expressions, etc.), evaluate the individual's Big Five personality traits and provide specific scores from 0-1.
    - **Openness**: Assess the individual's curiosity, imagination, and acceptance of new things.
    - **Conscientiousness**: Assess the individual's sense of responsibility, organizational skills, and self-discipline.
    - **Extraversion**: Assess the individual's sociability, energy, and optimism.
    - **Agreeableness**: Assess the individual's cooperativeness, empathy, and trustworthiness.
    - **Neuroticism**: Assess the individual's emotional stabilit

# 计算模型准确性参数

In [30]:
test_mae = compute_mae_by_index(0)
print(f"{(1-test_mae)*100}%")

90.18%


In [31]:
from tqdm import tqdm
import math

total_mae = 0.0
valid_samples = 0
mae_list = []
# 添加每个个性的MAE统计
trait_mae = {"Openness": 0.0, "Conscientiousness": 0.0, "Extraversion": 0.0, "Agreeableness": 0.0, "Neuroticism": 0.0}
trait_errors = {trait: [] for trait in trait_mae}
# 创建进度条对象
with tqdm(total=len(final_dataset['test']), 
         desc="评估进度",
         unit="样本",
         bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]") as pbar:

    for idx in range(len(final_dataset['test'])):
        prompt = alpaca_prompt.format(final_dataset['test'][idx]['input'],' ')
        prompt_encoding = tokenizer(prompt, return_tensors="pt").to(model.device)
        outputs = model.generate(
            **prompt_encoding,
            use_cache=True,
            max_new_tokens=128,
            temperature=0.85,
            top_k=50,
            top_p=0.9
        )
        generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        try:
            ai_scores = get_scores_from_text(generated_text[0])  # 假设有这个方法
            real_scores = get_scores_from_text(final_dataset['test'][idx]['output'])
            mae = calculate_mae(ai_scores, real_scores)
            if not math.isnan(mae):
                total_mae += mae
                valid_samples += 1
                mae_list.append(mae)
                # 更新每个个性的MAE
                for trait in ai_scores:
                    error = abs(ai_scores[trait] - real_scores[trait])
                    trait_mae[trait] += error
                    trait_errors[trait].append(error)
                
                # 更新进度条
                pbar.set_postfix({
                    '当前MAE': f"{mae:.4f}",
                    '平均MAE': f"{total_mae/valid_samples:.4f}",
                    '有效样本': valid_samples
                })
                
        except Exception as e:
            pbar.write(f"发生错误: {str(e)}")  # 使用pbar.write避免干扰进度条
        finally:
            pbar.update(1)  # 确保每次迭代都更新进度条

# 最终计算结果
if valid_samples > 0:
    final_mae = total_mae / valid_samples
    accuracy = (1 - final_mae) * 100
    variance = np.var(mae_list) if mae_list else 0
    # 计算每个个性的平均MAE
    for trait in trait_mae:
        trait_mae[trait] /= valid_samples
    trait_variances = {trait: np.var(errors) if errors else 0 for trait, errors in trait_errors.items()}
    print(f"\n{'='*60}")
    print(f"最终测试集平均MAE: {final_mae:.4f}")
    print(f"正确率: {accuracy:.2f}%")
    print(f"有效样本/总样本: {valid_samples}/{len(final_dataset['test'])}")
    print(f"方差: {variance:.6f}")
    print("\n各个性MAE及方差:")
    for trait, mae in trait_mae.items():
        print(f"{trait}: MAE={mae:.4f}, 方差={trait_variances[trait]:.6f}")
    print('='*60)
else:
    print("警告：没有有效样本可计算MAE")

评估进度: 100%|█████████████████████████████████████████████████████████████████████████████████| 300/300 [10:40<00:00]


最终测试集平均MAE: 0.1297
正确率: 87.03%
有效样本/总样本: 300/300
方差: 0.005266

各个性MAE及方差:
Openness: MAE=0.1348, 方差=0.010308
Conscientiousness: MAE=0.1336, 方差=0.009563
Extraversion: MAE=0.1370, 方差=0.010619
Agreeableness: MAE=0.1159, 方差=0.008191
Neuroticism: MAE=0.1274, 方差=0.009132


# 保存模型

In [32]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model\\tokenizer_config.json',
 'lora_model\\special_tokens_map.json',
 'lora_model\\tokenizer.json')